In [1]:
'''
Deep Learning Programming Assignment 2
--------------------------------------
Name: Vishnu Dutt Sharma
Roll No.: 12EC35018


======================================

Problem Statement:
This assignment has two problems.

P1. Implement a simple 1 or 2 hidden layer MLP USING any deep learning library
for predicting MNIST images.
P2. Implement a simple CNN USING any deep learning library
for predicting MNIST images.

Resources:
1. https://www.tensorflow.org/get_started/mnist/beginners
2. https://www.tensorflow.org/get_started/mnist/pros

======================================

Instructions:
1. Download the MNIST dataset from http://yann.lecun.com/exdb/mnist/
    (four files).
2. Extract all the files into a folder named `data' just outside
    the folder containing the main.py file. This code reads the
    data files from the folder '../data'.
3. Complete the functions in the train_dense.py and train_cnn.py files. You might also
    create other functions for your convenience, but do not change anything
    in the main.py file or the function signatures of the train and test
    functions in the train files.
4. The train function must train the neural network given the training
    examples and save the in a folder named `weights' in the same
    folder as main.py
5. The test function must read the saved weights and given the test
    examples it must return the predicted labels.
6. Submit your project folder with the weights. Note: Don't include the
    data folder, which is anyway outside your project folder.

Submission Instructions:
1. Fill your name and roll no in the space provided above.
2. Name your folder in format <Roll No>_<First Name>.
    For example 12CS10001_Rohan
3. Submit a zipped format of the file (.zip only).
'''

"\nDeep Learning Programming Assignment 2\n--------------------------------------\nName: Vishnu Dutt Sharma\nRoll No.: 12EC35018\n\n\n======================================\n\nProblem Statement:\nThis assignment has two problems.\n\nP1. Implement a simple 1 or 2 hidden layer MLP USING any deep learning library\nfor predicting MNIST images.\nP2. Implement a simple CNN USING any deep learning library\nfor predicting MNIST images.\n\nResources:\n1. https://www.tensorflow.org/get_started/mnist/beginners\n2. https://www.tensorflow.org/get_started/mnist/pros\n\n======================================\n\nInstructions:\n1. Download the MNIST dataset from http://yann.lecun.com/exdb/mnist/\n    (four files).\n2. Extract all the files into a folder named `data' just outside\n    the folder containing the main.py file. This code reads the\n    data files from the folder '../data'.\n3. Complete the functions in the train_dense.py and train_cnn.py files. You might also\n    create other functions for

In [2]:
import numpy as np
import os
# import train_dense
# import train_cnn
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import requests

In [14]:
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


def train_nn(trainX, trainY):
    
    trainX_mod = trainX.reshape((trainX.shape[0], trainX.shape[1]*trainX.shape[2]))
#     trainX_mod = trainX_mod - trainX_mod.mean(axis=0)
    y_mod = []
    for i in trainY:
        zer = [0]*10
        zer[i] = 1
        y_mod.append(zer)

    trainY_mod = np.asarray(y_mod)
    
    n_hidden = 400
    n_input = 784
    n_classes = 10

    n_hidden_1 = 400  # 1st layer number of features
    n_hidden_2 = 100# 2nd layer number of features
    n_input = trainX_mod.shape[1] # MNIST data input (img shape: 28*28)
    n_classes = trainY_mod.shape[1] # MNIST total classes (0-9 digits)

    
    
    
    X = tf.placeholder("float", shape=[None, n_input])
    y = tf.placeholder("float", shape=[None, n_classes])

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], 0.001)),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], 0.001)),
        'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], 0.001))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1], 0.001)),
        'b2': tf.Variable(tf.random_normal([n_hidden_2], 0.001)),
        'out': tf.Variable(tf.random_normal([n_classes], 0.001))
    }
    
    x = tf.placeholder(tf.float32, shape=[None, n_input])
    y_ = tf.placeholder(tf.float32, [None, 10])
    
    predict = multilayer_perceptron(X, weights, biases)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=y))
    updates = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    
    accuracy = 0.0
    batch_size = 100
    iterat = 20

    
    for epoch in range(iterat):
        train_X, test_X, train_y, test_y = train_test_split(trainX_mod, trainY_mod, test_size=0.20, random_state=random.randint(1,99))
        # Train with each example
        for i in range(int(len(train_X)/batch_size)):
            sess.run(updates, feed_dict={X: train_X[i*batch_size: (i + 1)*batch_size], y: train_y[i*batch_size: (i + 1)*batch_size]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 np.argmax(sess.run(predict, feed_dict={X: train_X, y: train_y}), axis=1))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 np.argmax(sess.run(predict, feed_dict={X: test_X, y: test_y}), axis=1))

        accuracy += test_accuracy
        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
    
    
    
    if not os.path.exists('./weights/'):
        os.makedirs('./weights/')
    
    w1= sess.run(weights['h1'])
    np.save('./weights/weight_nn_1', w1)

    w2= sess.run(weights['h2'])
    np.save('./weights/weight_nn_2', w2)
    
    wout= sess.run(weights['out'])
    np.save('./weights/weight_nn_out', wout)

    b1= sess.run(biases['b1'])
    np.save('./weights/bias_nn_1', b1)

    b2= sess.run(biases['b2'])
    np.save('./weights/bias_nn_2', b2)
    
    bout= sess.run(biases['out'])
    np.save('./weights/bias_nn_out', bout)
    
    sess.close()

    
def test_nn(testX):
    testX_mod = testX.reshape((testX.shape[0], testX.shape[1]*testX.shape[2]))
#     testX_mod = testX_mod - testX_mod.mean(axis=0)
    
    n_input = 784
    X = tf.placeholder("float", shape=[None, n_input])
    
    weights = dict()
    biases = dict()
    
    download = False
    if download:
        downloadData_CNN()
    

    weights['h1'] = np.load('./weights/weight_nn_1.npy')
    weights['h2'] = np.load('./weights/weight_nn_2.npy')
    weights['out'] = np.load('./weights/weight_nn_out.npy')

    biases['b1'] = np.load('./weights/bias_nn_1.npy')
    biases['b2'] = np.load('./weights/bias_nn_2.npy')
    biases['out'] = np.load('./weights/bias_nn_out.npy')

    x = tf.placeholder(tf.float32, shape=[None, n_input])
    
    predict = multilayer_perceptron(X, weights, biases)
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    
    output = np.argmax(sess.run(predict, feed_dict={X: testX_mod}), axis=1)
    
    return output
    

In [4]:
def load_mnist():
    data_dir = '../data'

    fd = open(os.path.join(data_dir, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trY = loaded[8:].reshape((60000)).astype(np.int)

    fd = open(os.path.join(data_dir, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teX = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teY = loaded[8:].reshape((10000)).astype(np.int)

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    perm = np.random.permutation(trY.shape[0])
    trX = trX[perm]
    trY = trY[perm]

    perm = np.random.permutation(teY.shape[0])
    teX = teX[perm]
    teY = teY[perm]

    return trX, trY, teX, teY

In [5]:
def print_digit(digit_pixels, label='?'):
    for i in range(28):
        for j in range(28):
            if digit_pixels[i, j] > 128:
                print('#',end='')
            else:
                print('.',end='')
        print('')

    print('Label: ', label)

In [6]:
def loadFromURL(outfile):
    http_proxy  = "http://10.3.100.207:8080"
    https_proxy = "https://10.3.100.207:8080"
    ftp_proxy   = "ftp://10.3.100.207:8080"

    proxyDict = { 
                  "http"  : http_proxy, 
                  "https" : https_proxy, 
                  "ftp"   : ftp_proxy
                }

    base = 'https://raw.githubusercontent.com/VishnuDuttSharma/DL_weights/master/asg3/'
    url  = base + outfile
    stream = requests.get(url, proxyDict)
    data = np.load(io.BytesIO(stream.content))
    
    return data

def downloadData():
    http_proxy  = "http://10.3.100.207:8080"
    https_proxy = "https://10.3.100.207:8080"
    ftp_proxy   = "ftp://10.3.100.207:8080"

    proxyDict = { 
                  "http"  : http_proxy, 
                  "https" : https_proxy, 
                  "ftp"   : ftp_proxy
                }
    
    base = 'https://raw.githubusercontent.com/VishnuDuttSharma/DL_weights/master/asg3/'
    
    weightList = ['bias_nn_1.npy', 'bias_nn_2.npy', 'bias_nn_out.npy', 'b_conv1.npy', 'b_conv2.npy', 'b_fc1.npy', 'b_fc2.npy', 'weight_nn_1.npy', 'weight_nn_2.npy', 'weight_nn_out.npy', 'W_conv1.npy', 'W_conv2.npy', 'W_fc1.npy', 'W_fc2.npy']
    
    if not os.path.exists('./weights/'):
        os.makedirs('./weights/')
    
    count = 0
    total = len(weightList)
    
    print('Downloading Weights')
    for name in weightList:
        url = base + name
        stream = requests.get(url, proxyDict)
        np_f = open('./weights/'+name, 'wb')
        np_f.write(stream.content)
        np_f.close()
        print(count+1,'/',total,' Complete')
        count += 1
        
    print('Download Complete')

def downloadData_DNN():
    http_proxy  = "http://10.3.100.207:8080"
    https_proxy = "https://10.3.100.207:8080"
    ftp_proxy   = "ftp://10.3.100.207:8080"

    proxyDict = { 
                  "http"  : http_proxy, 
                  "https" : https_proxy, 
                  "ftp"   : ftp_proxy
                }
    
    base = 'https://raw.githubusercontent.com/VishnuDuttSharma/DL_weights/master/asg3/'
    
    weightList = ['bias_nn_1.npy', 'bias_nn_2.npy', 'bias_nn_out.npy', 'weight_nn_1.npy', 'weight_nn_2.npy', 'weight_nn_out.npy']
    
    if not os.path.exists('./weights/'):
        os.makedirs('./weights/')
    
    count = 0
    total = len(weightList)
    
    print('Downloading DNN Weights')
    for name in weightList:
        url = base + name
        stream = requests.get(url, proxyDict)
        np_f = open('./weights/'+name, 'wb')
        np_f.write(stream.content)
        np_f.close()
        print(count+1,'/',total,' Complete')
        count += 1
        
    print('Download Complete')

def downloadData_CNN():
    http_proxy  = "http://10.3.100.207:8080"
    https_proxy = "https://10.3.100.207:8080"
    ftp_proxy   = "ftp://10.3.100.207:8080"

    proxyDict = { 
                  "http"  : http_proxy, 
                  "https" : https_proxy, 
                  "ftp"   : ftp_proxy
                }

    base = 'https://raw.githubusercontent.com/VishnuDuttSharma/DL_weights/master/asg3/'
    
    weightList = ['b_conv1.npy', 'b_conv2.npy', 'b_fc1.npy', 'b_fc2.npy',  'W_conv1.npy', 'W_conv2.npy', 'W_fc1.npy', 'W_fc2.npy']
    
    if not os.path.exists('./weights/'):
        os.makedirs('./weights/')
    
    count = 0
    total = len(weightList)
    
    print('Downloading CNN Weights')
    for name in weightList:
        url = base + name
        stream = requests.get(url, proxyDict)
        np_f = open('./weights/'+name, 'wb')
        np_f.write(stream.content)
        np_f.close()
        print(count+1,'/',total,' Complete')
        count += 1
        
    print('Download Complete')


In [28]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.001)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def train_cnn_web(trainX, trainY):
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    
    x = tf.placeholder(tf.float32, [None,28,28,1])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    
    y_mod = []
    for i in trainY:
        zer = [0]*10
        zer[i] = 1
        y_mod.append(zer)

    train_Y_mod = np.asarray(y_mod)
    
    
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

    train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    
    # Initializing the variables
    init = tf.global_variables_initializer()
    
    sess = tf.InteractiveSession()
    sess.run(init)
    
    
    
    epoch = 0
    batch_size = 100
    
    
    while epoch < 2:
        step = 1
        train_X, test_X, train_y, test_y = train_test_split(trainX, train_Y_mod, test_size=0.20, random_state=random.randint(1,99))
        # Keep training until reach max iterations
        for k in range(int(len(train_X)/batch_size)):
#             print("Range: ", k*batch_size, " to ", (k+1)*batch_size)
            
            batch_x, batch_y = train_X[k*batch_size : (k+1)*batch_size], train_y[k*batch_size : (k+1)*batch_size]
    
            train_step.run(feed_dict={x: batch_x, y_: batch_y, keep_prob: 0.6})
            train_accuracy = accuracy.eval(feed_dict={x:batch_x, y_: batch_y, keep_prob: 1.0})
            
            if step % 100 == 0:
                print("Epoch: %d, step %d, training accuracy %g"%(epoch, k, train_accuracy*100))
            
            
            step += 1
        
        acc = 0.0
        for k in range(int(len(test_X)/batch_size)):
            batch_x, batch_y = test_X[k*batch_size : (k+1)*batch_size], test_y[k*batch_size : (k+1)*batch_size]
            acc += sess.run(accuracy, feed_dict={x: batch_x, y_: batch_y, keep_prob: 1.0})
        
        print(" test accuracy, ", str(100*acc/int(len(test_X)/batch_size)))
        
        epoch += 1
            
    
    np.save('./weights/W_conv1', sess.run(W_conv1))
    np.save('./weights/W_conv2', sess.run(W_conv2))
    np.save('./weights/W_fc1', sess.run(W_fc1))
    np.save('./weights/W_fc2', sess.run(W_fc2))
    
    np.save('./weights/b_conv1', sess.run(b_conv1))
    np.save('./weights/b_conv2', sess.run(b_conv2))
    np.save('./weights/b_fc1', sess.run(b_fc1))
    np.save('./weights/b_fc2', sess.run(b_fc2))

    sess.close()
    
    
def test_cnn_web(testX):
    
    download = False
    if download:
        downloadData_CNN()
    
    W_conv1 = np.load('./weights/W_conv1.npy')
    b_conv1 = np.load('./weights/b_conv1.npy')

    W_conv2 = np.load('./weights/W_conv2.npy')
    b_conv2 = np.load('./weights/b_conv2.npy')

    W_fc1 = np.load('./weights/W_fc1.npy')
    b_fc1 = np.load('./weights/b_fc1.npy')

    W_fc2 = np.load('./weights/W_fc2.npy')
    b_fc2 = np.load('./weights/b_fc2.npy')
        
    x = tf.placeholder(tf.float32, [None,28,28,1])
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
        
    # Initializing the variables
    init = tf.global_variables_initializer()
    
    sess = tf.InteractiveSession()
    sess.run(init)
    
    output = []
    batch_size = 100
    
    for k in range((int(len(testX)/batch_size))):
        out = sess.run(y_conv, feed_dict={x: testX[k*batch_size : (k+1)*batch_size], keep_prob: 1.0})
        output += list(sess.run(tf.arg_max(out,1)))
        
        
    sess.close()
    
    return output
    

In [18]:
def main():
    trainX, trainY, testX, testY = load_mnist()
    print("Shapes: ", trainX.shape, trainY.shape, testX.shape, testY.shape)

#     print("\nDigit sample")
#     print_digit(trainX[1], trainY[1])

#     downloadData()
    
#     train_nn(trainX, trainY)
#     labels = test_nn(testX)
#     accuracy = np.mean((labels == testY)) * 100.0
#     print("\nDNN Test accuracy: %lf%%" % accuracy)


    train_cnn_web(trainX, trainY)
    labels = test_cnn_web(testX)
    accuracy = np.mean((labels == testY)) * 100.0
    print("\nCNN Test accuracy: %lf%%" % accuracy)

In [29]:
if __name__ == '__main__':
    main()

Shapes:  (60000, 28, 28, 1) (60000,) (10000, 28, 28, 1) (10000,)
Epoch: 0, step 99, training accuracy 95
Epoch: 0, step 199, training accuracy 97
Epoch: 0, step 299, training accuracy 99
Epoch: 0, step 399, training accuracy 99
 test accuracy,  98.208331267
Epoch: 1, step 99, training accuracy 100
Epoch: 1, step 199, training accuracy 99
Epoch: 1, step 299, training accuracy 98
Epoch: 1, step 399, training accuracy 98
 test accuracy,  98.7666645646

CNN Test accuracy: 98.590000%
